In [12]:
import json
import time
import heapq

def parse_json(input):
    data = json.loads(input)
    locations = data["Locations"]
    roads = data["Roads"]
    start_location = data["StartLocation"]
    budget = data["Budget"]
    map = {}
    for i in range(len(roads["a"])):
        a = roads["a"][i]
        b = roads["b"][i]
        price = roads["price"][i]
        if a not in map:
            map[a] = []
        if b not in map:
            map[b] = []
        map[a].append((b, price))
        map[b].append((a, price))
    return locations["critical"], start_location, budget, map

def possible_moves(location, budget, map, occupied_locations):
    moves = []
    for (next_location, cost) in map[location]:
        if next_location not in occupied_locations and cost <= budget:
            moves.append(next_location)
    return moves

def play_game(filename):
    with open(filename, 'r') as f:
        input_str = f.read()
    critical_locations, start_location, budget, map = parse_json(input_str)

    player = None
    computer = None
    
    current_location_player = start_location
    current_location_computer = start_location
    visted_locations_computer = []

    occupied_locations = []

    path_computer = []

    moves = []

    budget_player = budget
    budget_computer = budget

    current_playing = "1"

    print("Choose the player who you want to play White (1) or black (2):")
    input_choice = input()
    while input_choice not in ["1", "2"]:
        print(f"Invalid input '{input_choice}'. Choose the player who you want to play White (1) or black (2):")
        input_choice = input()

    if input_choice == "1":
        player = "1"
        computer = "2"
    else:
        player = "2"
        computer = "1"

    if player == "1":
        print(f"Game starts. player plays White and computer plays Black.")
    else:
        print(f"Game starts. player plays Black and computer plays White.")

    while True:
        if current_location_player == 88:
            if player == "1":
                print("player computer should get another play to make a draw")

                print("Computer is thinking...")
                time.sleep(2)  # Simulate some thinking time
            
                path_computer = algorithm(current_location_computer, 88, budget_computer, map, occupied_locations)
                next_location_computer = path_computer[1]
                if next_location_computer is None:
                    print("Computer has no moves left. Player wins!")

                if next_location_computer in critical_locations:
                    occupied_locations.append(next_location_computer)
                    visted_locations_computer.append(next_location_computer)
                else:
                    current_location_computer = next_location_computer
                    visted_locations_computer.append(next_location_computer)
                print(f"Computer moves to location {next_location_computer}.") 
                if current_location_computer == 88:
                    print(f"Draw!!!")
                else:
                    print("Player won!!!")
                return
            else:
                print("player won!!!")

                return

        elif current_location_computer == 88:
            if computer == "1":
                print("player should get another play to make a draw")
                moves = []
                print(f"turn for player, make your choice:")
                moves = possible_moves(current_location_player, budget_player, map, occupied_locations)
                print(f"Possible moves: {', '.join(str(loc) for loc in moves)}")
                print("You have 30 seconds to make a move.")
                start_time = time.time()

                while time.time() - start_time <= 30:
                    next_location_player = input(f"Enter next move for player {player}: ")
                    if not next_location_player.isdigit() or int(next_location_player) not in moves:
                        print(f"Invalid move '{next_location_player}'. You can move to any of the following locations: {', '.join(str(loc) for loc in moves)}")
                    else:
                        if int(next_location_player) in critical_locations:
                            occupied_locations.append(int(next_location_player))
                            budget_player -= next((cost for (loc, cost) in map[current_location_player] if loc == int(next_location_player)), None)
                            print(f"player location is currently: {next_location_player}")
                            break
                        else:
                            budget_player -= next((cost for (loc, cost) in map[current_location_player] if loc == int(next_location_player)), None)
                            current_location_player = int(next_location_player)
                            print(f"player location is currently: {next_location_player}")
                            break
                if current_location_player == 88:
                    print("Draw!!!")
                else:
                    print("Computer wins!!!")
                return
            else:
                print("Computer won")
                return


        if current_playing == player:
            current_playing = computer
            moves = []
            print(f"turn for player, make your choice:")
            moves = possible_moves(current_location_player, budget_player, map, occupied_locations)
            print(f"Possible moves: {', '.join(str(loc) for loc in moves)}")
            print("You have 30 seconds to make a move.")
            start_time = time.time()

            while time.time() - start_time <= 30:
                next_location_player = input(f"Enter next move for player {player}: ")
                if not next_location_player.isdigit() or int(next_location_player) not in moves:
                    print(f"Invalid move '{next_location_player}'. You can move to any of the following locations: {', '.join(str(loc) for loc in moves)}")
                else:
                    if int(next_location_player) in critical_locations:
                        occupied_locations.append(int(next_location_player))
                        budget_player -= next((cost for (loc, cost) in map[current_location_player] if loc == int(next_location_player)), None)
                        print(f"player location is currently: {next_location_player}")
                        break
                    else:
                        budget_player -= next((cost for (loc, cost) in map[current_location_player] if loc == int(next_location_player)), None)
                        current_location_player = int(next_location_player)
                        print(f"player location is currently: {next_location_player}")
                        print(f"The budget of player is now: {budget_player}")          
                        break
            print("Time's up! Skipping turn.")
            
        else:
            current_playing = player
            print("Computer is thinking...")
            time.sleep(1)  # Simulate some thinking time
            path_computer = algorithm(current_location_computer, 88, budget_computer, map, occupied_locations)
            print(f"{path_computer}")
            current_location_computer = path_computer[1]
            next_location_computer = path_computer[1]
            #print (f"{next_location_computer}")
            if next_location_computer is None:
                print("Computer has no moves left. Player wins!")
                
            if next_location_computer in critical_locations:
                
                occupied_locations.append(next_location_computer)
                cost = next((cost for (loc, cost) in map[current_location_computer] if loc == int(next_location_computer)), None)
                if cost is None:
                    print("No matching location found in map")
                else:
                    budget_computer -= cost
                    current_location_computer = next_location_computer
                visted_locations_computer.append(next_location_computer)            
            else:
                cost = next((cost for (loc, cost) in map[current_location_computer] if loc == int(next_location_computer)), None)
            if cost is None:
                print("No matching location found in map")
            else:
                budget_computer -= cost
                current_location_computer = next_location_computer
            visted_locations_computer.append(next_location_computer)            
            print(f"Computer moves to location {next_location_computer}.")  
            print(f"The budget of computer is now: {budget_computer}")          


def algorithm(begin, end, budgetComputer, map, occupied_locations):
    distances = {node: float('inf') for node in map}
    distances[begin] = 0
    queue = [(0, begin, [begin])]
    while queue:
        current_distance, current_node, path = heapq.heappop(queue)
        if current_distance > distances[current_node]:
            continue
        if current_node == end:
            return path
        for next_node, cost in map[current_node]:
            if next_node in occupied_locations:
                continue
            if budgetComputer - cost >= 0:
                new_path = path + [next_node]
                heapq.heappush(queue, (current_distance + cost, next_node, new_path))
            if current_distance + cost < distances[next_node]:
                distances[next_node] = current_distance + cost
                new_path = path + [next_node]
                heapq.heappush(queue, (current_distance + cost, next_node, new_path))
    return None

''' first algorithm I designed
    def algorithm1(location, budgetComputer, map, occupied_locations):
    # Simple algorithm: just choose the cheapest possible move that hasn't been taken yet
    moves = possible_moves(location, budgetComputer, map, occupied_locations)
    if not moves:
        return None
    for move in moves:
        if move == 88 and budgetComputer - next((cost for (loc, cost) in map[location] if loc == int(move)), None) >= 0:
            return move
    cheapest_move = min(moves, key=lambda loc: next((cost for (next_loc, cost) in map[location] if next_loc == loc), float('inf')))
    return cheapest_move   ''' 
    
play_game('game6_medium.json')

Choose the player who you want to play White (1) or black (2):
Invalid input ''. Choose the player who you want to play White (1) or black (2):
Game starts. player plays Black and computer plays White.
Computer is thinking...
[26, 19, 13, 88]
No matching location found in map
Computer moves to location 19.
The budget of computer is now: 12
turn for player, make your choice:
Possible moves: 23, 10, 19, 8
You have 30 seconds to make a move.
player location is currently: 10
The budget of player is now: 11
Time's up! Skipping turn.
Computer is thinking...
[19, 13, 88]
No matching location found in map
Computer moves to location 13.
The budget of computer is now: 12
turn for player, make your choice:
Possible moves: 25, 26, 88
You have 30 seconds to make a move.
player location is currently: 88
The budget of player is now: 8
Time's up! Skipping turn.
player won!!!
